In [7]:
import nltk
nltk.download('vader_lexicon')
nltk.download('punkt')

import os
import openai
from dotenv import load_dotenv
import json

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

huggingface_token = os.getenv('HF_TOKEN')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\stx6\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\stx6\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()

text = '''A Chinese warship cut off an American warship in the Taiwan Strait while both U.S. and Chinese defense chiefs attended the same conference in Singapore and  days after a Chinese fighter jet "thumped" a U.S. Air Force RC-135 in the South China Sea.'''

score = sia.polarity_scores(text)
print(score)

{'neg': 0.126, 'neu': 0.803, 'pos': 0.071, 'compound': -0.34}


In [5]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import word_tokenize

sia = SentimentIntensityAnalyzer()

# Define your list of extreme negative words
extreme_neg_words = ["fuck"]

def detect_neg_phrases(text):
    words = word_tokenize(text)
    neg_phrases = []

    for i in range(len(words) - 1):
        phrase = f"{words[i]} {words[i + 1]}"
        if sia.polarity_scores(phrase)["compound"] < -0.5 and (words[i] in extreme_neg_words or words[i + 1] in extreme_neg_words):
            neg_phrases.append(phrase)

    return neg_phrases

text = "I would like to fuck your mother"
print(detect_neg_phrases(text))

['to fuck', 'fuck your']


In [7]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": 'perform sentiment analysis and return me the extremely offensive text/phrases without any explanation, in the json format: {"detected: [<phrase>, <phrase>] "} '},
        {"role": "user", "content": '''Apple Inc. decided to hire John Smith as their new CEO. The event will take place in San Francisco on 2022-07-23.'''},
        # {"role": "assistant", "content": 'Please identify and categorize the entities in the provided text.'},
    ]
)

print(response['choices'][0]['message']['content'])

{"detected": []}


In [18]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()

text = '''This idiot demands 50 Patriot3 systems from 'allies'.  Also demands good fighter jets otherwise counter offensive hard to do because lack of air cover '''

score = sia.polarity_scores(text)
print(score)

{'neg': 0.308, 'neu': 0.554, 'pos': 0.138, 'compound': -0.6705}


In [9]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": 'perform sentiment analysis and return me the extremely offensive text/phrases without any explanation, in the json format: {"detected: [<phrase>, <phrase>] "} '},
        {"role": "user", "content": '''"I would like to fuck your mother"'''},
        # {"role": "assistant", "content": 'Please identify and categorize the entities in the provided text.'},
    ]
)

print(response['choices'][0]['message']['content'])

{"detected": ["fuck your mother"]}


In [10]:
response

<OpenAIObject chat.completion id=chatcmpl-7QAUS5QzenX4MVKvbyZFX72eImTBZ at 0x26a98b6fdd0> JSON: {
  "id": "chatcmpl-7QAUS5QzenX4MVKvbyZFX72eImTBZ",
  "object": "chat.completion",
  "created": 1686471208,
  "model": "gpt-3.5-turbo-0301",
  "usage": {
    "prompt_tokens": 54,
    "completion_tokens": 9,
    "total_tokens": 63
  },
  "choices": [
    {
      "message": {
        "role": "assistant",
        "content": "{\"detected\": [\"fuck your mother\"]}"
      },
      "finish_reason": "stop",
      "index": 0
    }
  ]
}

In [13]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": 'perform sentiment analysis and return me the extremely offensive text/phrases without any explanation, in the json format: {"detected: [<phrase>, <phrase>] "} '},
        {"role": "user", "content": 'A Chinese warship cut off an American warship in the Taiwan Strait while both U.S. and Chinese defense chiefs attended the same conference in Singapore and  days after a Chinese fighter jet "thumped" a U.S. Air Force RC-135 in the South China Sea.'},
        # {"role": "assistant", "content": 'Please identify and categorize the entities in the provided text.'},
    ]
)

print(response['choices'][0]['message']['content'])

{"detected": ["cut off", "thumped"]}


In [6]:
import requests

huggingface_token = os.getenv('HF_TOKEN')
API_URL = "https://api-inference.huggingface.co/models/unitary/unbiased-toxic-roberta"
headers = {"Authorization": f"Bearer {huggingface_token}"}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()

output = query({
	"inputs": 'A Chinese warship cut off an American warship in the Taiwan Strait while both U.S. and Chinese defense chiefs attended the same conference in Singapore and  days after a Chinese fighter jet "thumped" a U.S. Air Force RC-135 in the South China Sea.',
})
output

[[{'label': 'toxicity', 'score': 0.0006405769963748753},
  {'label': 'male', 'score': 0.00028635511989705265},
  {'label': 'white', 'score': 0.00023706024512648582},
  {'label': 'insult', 'score': 0.000138996314490214},
  {'label': 'identity_attack', 'score': 0.00010856699373107404},
  {'label': 'christian', 'score': 6.47075503366068e-05},
  {'label': 'female', 'score': 6.121212209109217e-05},
  {'label': 'muslim', 'score': 5.820589285576716e-05},
  {'label': 'black', 'score': 5.386660268413834e-05},
  {'label': 'psychiatric_or_mental_illness', 'score': 4.63088508695364e-05},
  {'label': 'threat', 'score': 3.526092768879607e-05},
  {'label': 'obscene', 'score': 3.0274881282821298e-05},
  {'label': 'sexual_explicit', 'score': 2.420160308247432e-05},
  {'label': 'homosexual_gay_or_lesbian', 'score': 2.0228162611601874e-05},
  {'label': 'jewish', 'score': 1.9041299310629256e-05},
  {'label': 'severe_toxicity', 'score': 1.1434019597800216e-06}]]

[[{'label': 'toxicity', 'score': 0.0005681620095856488},
  {'label': 'male', 'score': 0.00018487480701878667},
  {'label': 'insult', 'score': 0.0001848305546445772},
  {'label': 'psychiatric_or_mental_illness', 'score': 0.00010473166184965521},
  {'label': 'female', 'score': 0.00010348340583732352},
  {'label': 'christian', 'score': 8.914023783290759e-05},
  {'label': 'muslim', 'score': 4.75365886813961e-05},
  {'label': 'threat', 'score': 4.241721035214141e-05},
  {'label': 'obscene', 'score': 3.9702561480225995e-05},
  {'label': 'white', 'score': 3.022039345523808e-05},
  {'label': 'sexual_explicit', 'score': 2.5585730327293277e-05},
  {'label': 'identity_attack', 'score': 2.26132724492345e-05},
  {'label': 'homosexual_gay_or_lesbian', 'score': 2.148881867469754e-05},
  {'label': 'jewish', 'score': 1.827706364565529e-05},
  {'label': 'black', 'score': 1.7253853002330288e-05},
  {'label': 'severe_toxicity', 'score': 1.0941928394458955e-06}]]

NameError: name 'os' is not defined